# Week 4: Milestone 1 – Smart Tracking System Blockchain Ledger (Draft)

Follow these steps to verify Python↔Ganache↔Smart Contract connection.

1. Setup and Connection

1. **Open Jupyter Notebook**  
2. **Verify Ganache** is running (Desktop App “Quickstart Ethereum” or `ganache-cli`).  
3. **Check RPC port** (default is `7545` in Ganache Desktop).

Below we’ll connect Python → Ganache → our IoTDataStorage contract.

In [1]:
# Cell 1: imports & connection
from web3 import Web3
import json, os

# 1️⃣ Point to your Ganache RPC (update port if needed)
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2️⃣ Test connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


2. Load Contract ABI and Address

🔗 Now we load the **ABI** exported (in `/artifacts/IoTDataStorageABI.json`) and the **deployed address** from Remix.

In [2]:
# Cell 2: load ABI + contract
abi_path = os.path.join("..", "artifacts", "IoTDataStorageABI.json")
with open(abi_path) as f:
    abi = json.load(f)

# Replace with actual deployed address - convert to checksum format
contract_address_raw = "0x55e63ee6915c2c874b4adf68311a917201f719c0"
contract_address = Web3.to_checksum_address(contract_address_raw)

contract = web3.eth.contract(address=contract_address, abi=abi)

# Use the Ganache account that has a balance
web3.eth.default_account = web3.eth.accounts[1]

print(f"▶️ Using account {web3.eth.default_account}")
print(f"✅ Loaded contract at {contract_address}")

# Optional: Test the connection
try:
    total_package_records = contract.functions.getTotalPackages().call()
    print(f"📊 Current total records: {total_package_records}")
    print("🎉 Contract connection successful!")
except Exception as e:
    print(f"❌ Error testing contract connection: {e}")

▶️ Using account 0x68737979748dEAAF63d1b5D03151693128366D93
✅ Loaded contract at 0x55E63eE6915c2C874B4aDf68311a917201F719c0
📊 Current total records: 0
🎉 Contract connection successful!


3. Read-Only Call: getTotalRecords()

Let’s check that our contract is responding:

In [3]:
# Cell 3: read-only call
total_events = contract.functions.getTotalEvents().call()
print(f"🔢 Total stored entries on‐chain: {total_events}")

🔢 Total stored entries on‐chain: 0


4. Write a dummy entry

Store one dummy IoT record (TEST001) to prove transactions work.

In [4]:
# Cell 4: Store three package entries from CSV data
import datetime
from web3 import Web3

# First, make sure we have a valid account
print("Available accounts:")
for i, account in enumerate(web3.eth.accounts):
    balance = web3.eth.get_balance(account)
    print(f"  [{i}] {account} - Balance: {web3.from_wei(balance, 'ether')} ETH")

# Set the account explicitly (use account[0] which is usually the deployer)
sender_account = web3.eth.accounts[0]  # or web3.eth.accounts[1] if you prefer
print(f"\n▶️ Using account: {sender_account}")

# Check account balance
balance = web3.eth.get_balance(sender_account)
balance_eth = web3.from_wei(balance, 'ether')
print(f"💰 Account balance: {balance_eth} ETH")

if balance_eth < 0.1:
    print("⚠️ Warning: Account balance is low. You may need more ETH for gas fees.")
    print("💡 In Ganache, you can restart to reset balances or use a different account.")

# Helper function to convert datetime string to timestamp
def datetime_to_timestamp(datetime_str):
    if datetime_str and datetime_str.strip():
        dt = datetime.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
        return int(dt.timestamp())
    return 0

# Package data from your CSV
packages_data = [
    {
        "tracking_number": "Fed262013051",
        "package_type": "Express",
        "carrier": "FedEx",
        "weight_kg": 46.99,
        "length_cm": 75.15,
        "width_cm": 62.2,
        "height_cm": 73.04,
        "volume_cm3": 341413.06,
        "origin_city": "person person",
        "origin_state": "Iowa",
        "origin_zip": "98801",
        "destination_city": "person person",
        "destination_state": "Vermont",
        "destination_zip": "41150",
        "current_status": "In Transit",
        "estimated_delivery": "2025-05-05 23:08:24",
        "signature_required": False,
        "description": "Express package weighing 46.99kg, dimensions: 75.15x62.2x73.04cm, volume: 341413.06cm³"
    },
    {
        "tracking_number": "UPS915351342",
        "package_type": "Refrigerated",
        "carrier": "UPS",
        "weight_kg": 11.22,
        "length_cm": 14.42,
        "width_cm": 93.11,
        "height_cm": 31.56,
        "volume_cm3": 42373.91,
        "origin_city": "person person",
        "origin_state": "Tennessee",
        "origin_zip": "70115",
        "destination_city": "person personperson",
        "destination_state": "Mississippi",
        "destination_zip": "20469",
        "current_status": "Exception",
        "estimated_delivery": "2025-05-13 06:46:40",
        "signature_required": False,
        "description": "Refrigerated package weighing 11.22kg, dimensions: 14.42x93.11x31.56cm, volume: 42373.91cm³"
    },
    {
        "tracking_number": "UPS948274525",
        "package_type": "Overnight",
        "carrier": "UPS",
        "weight_kg": 6.66,
        "length_cm": 89.14,
        "width_cm": 97.28,
        "height_cm": 10.25,
        "volume_cm3": 88883.28,
        "origin_city": "personperson",
        "origin_state": "Indiana",
        "origin_zip": "71240",
        "destination_city": "person person",
        "destination_state": "Virginia",
        "destination_zip": "22212",
        "current_status": "In Transit",
        "estimated_delivery": "2025-05-16 22:43:59",
        "signature_required": True,
        "description": "Overnight package weighing 6.66kg, dimensions: 89.14x97.28x10.25cm, volume: 88883.28cm³"
    }
]

# Store each package
for i, pkg in enumerate(packages_data):
    try:
        print(f"\n📦 Storing package {i+1}/3: {pkg['tracking_number']}")

        # Convert weight from kg to grams (to avoid decimals in Solidity)
        weight_grams = int(pkg['weight_kg'] * 1000)

        # Convert volume to integer
        volume_cm3 = int(pkg['volume_cm3'])

        # Convert estimated delivery to timestamp
        estimated_delivery_timestamp = datetime_to_timestamp(pkg['estimated_delivery'])

        # Create origin and destination strings
        origin_location = f"{pkg['origin_city']}, {pkg['origin_state']}"
        destination_location = f"{pkg['destination_city']}, {pkg['destination_state']}"

        # First estimate gas needed
        try:
            estimated_gas = contract.functions.storePackage(
                pkg['tracking_number'],
                pkg['package_type'],
                pkg['carrier'],
                pkg['current_status'],
                weight_grams,
                volume_cm3,
                origin_location,
                destination_location,
                estimated_delivery_timestamp,
                pkg['signature_required']
            ).estimate_gas({'from': sender_account})

            # Add 20% buffer to estimated gas
            gas_limit = int(estimated_gas * 1.2)
            print(f"📊 Estimated gas: {estimated_gas}, Using: {gas_limit}")

        except Exception as gas_error:
            print(f"⚠️ Gas estimation failed: {gas_error}")
            gas_limit = 500_000  # Fallback to higher limit

        # Store the package with higher gas limit
        tx = contract.functions.storePackage(
            pkg['tracking_number'],        # tracking number
            pkg['package_type'],           # package type
            pkg['carrier'],                # carrier
            pkg['current_status'],         # current status
            weight_grams,                  # weight in grams
            volume_cm3,                    # volume in cubic cm
            origin_location,               # origin city
            destination_location,          # destination city
            estimated_delivery_timestamp,  # estimated delivery timestamp
            pkg['signature_required']      # signature required
        ).transact({
            "from": sender_account,
            "gas": gas_limit,
            "gasPrice": web3.to_wei("2", "gwei")  # Slightly higher gas price
        })

        print(f"📝 Transaction sent: {tx.hex()}")

        # Wait for transaction to be mined
        receipt = web3.eth.wait_for_transaction_receipt(tx)
        print(f"✅ Package {pkg['tracking_number']} stored successfully!")
        print(f"📊 Gas used: {receipt.gasUsed}")

    except Exception as e:
        print(f"❌ Error storing package {pkg['tracking_number']}: {e}")

# Verify all packages were stored
try:
    total_packages = contract.functions.getTotalPackages().call()
    print(f"\n🎯 Total packages stored: {total_packages}")

    # Test retrieving one of the packages
    if total_packages > 0:
        print(f"\n📋 Testing package retrieval for 'Fed262013051':")
        package_data = contract.functions.getPackage("Fed262013051").call()
        print(f"Tracking Number: {package_data[0]}")
        print(f"Package Type: {package_data[1]}")
        print(f"Carrier: {package_data[2]}")
        print(f"Status: {package_data[3]}")
        print(f"Weight (grams): {package_data[4]}")
        print(f"Volume (cm³): {package_data[5]}")
        print(f"Origin: {package_data[6]}")
        print(f"Destination: {package_data[7]}")

except Exception as e:
    print(f"❌ Error reading stored packages: {e}")

print(f"\n🎉 Package storage process completed!")

Available accounts:
  [0] 0x08Dd690bCBEB81b9f7b1AC0899B0E39c6545aB18 - Balance: 99.99706492671484375 ETH
  [1] 0x68737979748dEAAF63d1b5D03151693128366D93 - Balance: 100 ETH
  [2] 0x3C557cb51704502b51c75f1C2b20245439DBFeC3 - Balance: 100 ETH
  [3] 0xbf24cCD857B5B9d779B666ec18fde23fEFbe0B02 - Balance: 100 ETH
  [4] 0xa77C32FDdbB9a2A83AeA7E20E9561424448A2434 - Balance: 100 ETH
  [5] 0x2f76469B50810C92C181E1eFE7d4D59b519A471e - Balance: 100 ETH
  [6] 0xb6A8691d0067f991B808C1e13Bb558CC92CF1E2c - Balance: 100 ETH
  [7] 0xf916288b73fCD6a1946963aeeedB6d9a1eA0215c - Balance: 100 ETH
  [8] 0xc9c6222c96faCBC8A3767016F470b08765fe7293 - Balance: 100 ETH
  [9] 0xdcEf0F161098d9E0f172B95c3fC295cdEA4aa107 - Balance: 100 ETH

▶️ Using account: 0x08Dd690bCBEB81b9f7b1AC0899B0E39c6545aB18
💰 Account balance: 99.99706492671484375 ETH

📦 Storing package 1/3: Fed262013051
📊 Estimated gas: 293064, Using: 351676
📝 Transaction sent: dcd5a80734646ed9c2a7b967fa9d4e68a39a5712e161442fe55b1db1cd28590d
✅ Package Fed262

5. Verify it went through

After mining, call again to see the updated count and fetch the index.

In [5]:
# Cell 5: Verify storage with table display
import pandas as pd

# Get verification data
new_total_packages = contract.functions.getTotalPackages().call()
first_package = contract.functions.getPackage("Fed262013051").call()

# Create verification summary table
verification_data = {

    'Metric': ['Total Packages', 'Sample Package ID', 'Package Exists', 'Package Data Length'],
    'Value': [
        new_total_packages,
        'Fed262013051',
        'Yes' if first_package else 'No',
        len(first_package) if first_package else 0
    ]
}

verification_df = pd.DataFrame(verification_data)
print("📊 Storage Verification Summary:")
print("=" * 40)
display(verification_df)

# If package data exists, show detailed package information
if first_package:
    print("\n📦 Sample Package Details:")
    print("=" * 40)

    # Assuming the package data structure (adjust field names as needed)
    package_details = {
        'Field': [
            'trackingNumber',
            'packageType',
            'carrier',
            'currentStatus',
            'weight',
            'volume',
            'originCity',
            'destinationCity',
            'estimatedDelivery',
            'actualDelivery',
            'signatureRequired',
        ],
        'Value': [
            first_package[0] if len(first_package) > 0 else 'N/A',
            first_package[1] if len(first_package) > 1 else 'N/A',
            first_package[2] if len(first_package) > 2 else 'N/A',
            first_package[3] if len(first_package) > 3 else 'N/A',
            first_package[4] if len(first_package) > 4 else 'N/A',
            first_package[5] if len(first_package) > 5 else 'N/A',
            first_package[6] if len(first_package) > 6 else 'N/A',
            first_package[7] if len(first_package) > 7 else 'N/A',
            first_package[8] if len(first_package) > 8 else 'N/A',
            first_package[9] if len(first_package) > 9 else 'N/A',
            first_package[10] if len(first_package) > 10 else 'N/A'
        ]
    }

    package_df = pd.DataFrame(package_details)
    display(package_df)
else:
    print("\n⚠️ No package data found for ID: Fed262013051")

📊 Storage Verification Summary:


,Metric,Value
0,Total Packages,3
1,Sample Package ID,Fed262013051
2,Package Exists,Yes
3,Package Data Length,11



📦 Sample Package Details:


,Field,Value
0,trackingNumber,Fed262013051
1,packageType,Express
2,carrier,FedEx
3,currentStatus,In Transit
4,weight,46990
5,volume,341413
6,originCity,"person person, Iowa"
7,destinationCity,"person person, Vermont"
8,estimatedDelivery,1746457704
9,actualDelivery,0


In [24]:
# Cell 6: Adding New IoT Data using our Output (logistics_data.json) on the Blockchain
import pandas as pd
import json

# Load the JSON data
with open("../data/logistics_data.json", "r") as file:
    logistics_data = json.load(file)

# Convert to DataFrame for easier handling (if it's a list of JSON objects)
if isinstance(logistics_data, list):
    df = pd.DataFrame(logistics_data)
else:
    # If it's a single JSON object, convert to list first
    df = pd.DataFrame([logistics_data])

sender_account = web3.eth.accounts[0]
print(f"🔐 Using account: {sender_account}")
print("=" * 50)

# Create a list to store transaction results for table display
transaction_results = []

for i, row in df.iterrows():
    try:
        # Get tracking number (adjust field name based on your JSON structure)
        tracking_id = row.get('trackingNumber', f'package_{i}')

        # Convert entire row to JSON string for storage
        row_json = row.to_json()

        # trackingNumber✅
        # packageType✅
        # carrier✅
        # currentStatus✅
        # weight✅
        # volume✅
        # originCity✅
        # destinationCity✅
        # estimatedDelivery✅
        # actualDelivery✅
        # signatureRequired✅

        tx = contract.functions.storePackage(
            str(row.get('tracking_number')),     # Package Tracking ID
            str(row.get('package_type'),            # Package type
            str(row.get('carrier'),
            str(row.get('status'),
            int(float(row.get('dimensions', {}).get('weight')),  # Weight (uint256)
            int(float(row.get('dimensions', {}).get('volume')),  # Volume (uint256)
            str(row.get('origin', {}).get('city'), # Origin city
            str(row.get('destination', {}).get('city'), # Destination city
            str(row.get('estimated_delivery'),
            str(row.get('actual_delivery'),
            str(row.get('signature_required')
        ).transact({
            "from": sender_account,
            "gas": 300_000,
            "gasPrice": web3.to_wei("1", "gwei")
        })

        receipt = web3.eth.wait_for_transaction_receipt(tx)

        # Store result for table
        transaction_results.append({
            'Index': i+1,
            'Tracking ID': tracking_id,
            'Status': '✅ Success',
            'Gas Used': receipt.gasUsed,
            'Transaction Hash': tx.hex()[:12] + '...'  # Shortened for display
        })

        print(f"[{i+1}/{len(df)}] ✅ Stored package: {tracking_id} | Tx: {tx.hex()}")

    except Exception as e:
        # Store error result for table
        transaction_results.append({
            'Index': i+1,
            'Tracking Number': tracking_id if 'tracking_id' in locals() else f'package_{i}',
            'Status': '❌ Failed',
            'Gas Used': 0,
            'Transaction Hash': 'N/A'
        })

        print(f"[{i+1}/{len(df)}] ❌ Error storing package {tracking_id}: {e}")

# Display results in table format
print("\n📊 Transaction Summary:")
print("=" * 50)
results_df = pd.DataFrame(transaction_results)
display(results_df)

# Display summary statistics
successful_txs = len([r for r in transaction_results if '✅' in r['Status']])
failed_txs = len([r for r in transaction_results if '❌' in r['Status']])
total_gas_used = sum([r['Gas Used'] for r in transaction_results])

summary_data = {
    'Metric': ['Total Packages Processed', 'Successful Transactions', 'Failed Transactions', 'Success Rate', 'Total Gas Used'],
    'Value': [
        len(df),
        successful_txs,
        failed_txs,
        f"{(successful_txs/len(df)*100):.1f}%" if len(df) > 0 else "0%",
        f"{total_gas_used:,}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n📈 Processing Summary:")
print("=" * 50)
display(summary_df)

🔐 Using account: 0x6BCdD0F870A95d80fc9910C88C3F1435f8E33B87
[1/2000] ❌ Error storing package Fed262013051: {'message': 'VM Exception while processing transaction: revert Tracking number already exists', 'stack': 'RuntimeError: VM Exception while processing transaction: revert Tracking number already exists\n    at LegacyTransaction.fillFromResult (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:12745)\n    at Miner.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:36703)\n    at async Miner.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\ganache\\dist\\node\\1.js:2:35116)\n    at async Miner.mine (C:\\Program Files\\WindowsApps\\GanacheUI_2.7.1.0_x64__rb4352f0jd4m2\\app\\resources\\static\\node\\node_modules\\gan

,Index,Tracking ID,Status,Gas Used,Transaction Hash
0,1,Fed262013051,❌ Failed,0,N/A
1,2,UPS915351342,❌ Failed,0,N/A
2,3,UPS948274525,❌ Failed,0,N/A
3,4,USP11039782,❌ Failed,0,N/A
4,5,DHL456680458,❌ Failed,0,N/A
...,...,...,...,...,...
1995,1996,Ama864141175,❌ Failed,0,N/A
1996,1997,Fed611549996,❌ Failed,0,N/A
1997,1998,Fed958545778,❌ Failed,0,N/A
1998,1999,Fed421911834,❌ Failed,0,N/A



📈 Processing Summary:


,Metric,Value
0,Total Packages Processed,2000
1,Successful Transactions,20
2,Failed Transactions,1980
3,Success Rate,1.0%
4,Total Gas Used,"5,915,132"


In [18]:
# Read first row from CSV

# For this example we use the guide from camu which is the sampler data
first_record = contract.functions.getRecord(0).call()
print("CSV First Record (concatenated):")
# TEST003
# New York
# 22.5°C
print(f"  Package Name: {first_record[0]}")
print(f"  Location: {first_record[2]}")
print(f"  Temperature: {first_record[3]}")

# In this case we use our generated logistics data
# Fetch last record from blockchain
last_record = contract.functions.getRecord(100).call()

print("\nBlockchain Last Record:")
print(f"  Shipment ID: {last_record[0]}")
print(f"  Data Type: {last_record[1]}")
print(f"  Full Row Data: {last_record[2]}")


CSV First Record (concatenated):
  Package Name: 1749516561
  Location: New York
  Temperature: 22.5°C

Blockchain Last Record:
  Shipment ID: 1749516919
  Data Type: SHIP5841
  Full Row Data: FullRow


# Data Retrieval and Processing

In [16]:
total_package_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_package_records}")

Total IoT records stored: 101


In [26]:
import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_package_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "device_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })


# Convert to a DataFrame
df = pd.DataFrame(data)


# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")


# Display first few records
print(df.head())

            timestamp device_id data_type  \
0 2025-06-10 00:49:21   TEST003  New York   
1 2025-06-10 00:55:15  SHIP7734   FullRow   
2 2025-06-10 00:55:15  SHIP4309   FullRow   
3 2025-06-10 00:55:15  SHIP1802   FullRow   
4 2025-06-10 00:55:15  SHIP9967   FullRow   

                                          data_value  
0                                             22.5°C  
1  2025-05-18 15:46:54|2025-05-18 11:46:54|2025-0...  
2  2025-05-18 09:41:54|2025-05-18 06:41:54|2025-0...  
3  2025-05-18 13:45:54|2025-05-18 10:45:54|2025-0...  
4  2025-05-18 05:57:54|2025-05-18 01:57:54|2025-0...  


In [24]:

import numpy as np


# Extract numeric values from 'data_value' where applicable
df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)


# Handle missing values (if any)
df.fillna(0, inplace=True)


# Display cleaned data
print(df.head())

            timestamp device_id data_type  \
0 2025-06-10 00:49:21   TEST003  New York   
1 2025-06-10 00:55:15  SHIP7734   FullRow   
2 2025-06-10 00:55:15  SHIP4309   FullRow   
3 2025-06-10 00:55:15  SHIP1802   FullRow   
4 2025-06-10 00:55:15  SHIP9967   FullRow   

                                          data_value  numeric_value  
0                                             22.5°C           22.5  
1  2025-05-18 15:46:54|2025-05-18 11:46:54|2025-0...         2025.0  
2  2025-05-18 09:41:54|2025-05-18 06:41:54|2025-0...         2025.0  
3  2025-05-18 13:45:54|2025-05-18 10:45:54|2025-0...         2025.0  
4  2025-05-18 05:57:54|2025-05-18 01:57:54|2025-0...         2025.0  
